### 10alytics Data Engineering Capstone Project for Norby Inc. (Group 2) - Oct 2023

### Importing neccessary libraries

In [1]:
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
from time import time
import boto3

### Connecting to Data lake (Amazon S3 Bucket) 

In [2]:
#connecting to the client
s3_client = boto3.resource('s3')

#extracting data from orders fact table
client = boto3.client('s3')

bucket_name = '10alytics-capstone-bucket'
key ='orders_data/orders.csv'

response = client.get_object(Bucket=bucket_name,Key=key)
orders_data = response['Body'].read().decode('utf-8')

orders =pd.read_csv(StringIO(orders_data))

#extracting data from reviews fact table
client = boto3.client('s3')

bucket_name = '10alytics-capstone-bucket'
key ='orders_data/reviews.csv'

response = client.get_object(Bucket=bucket_name,Key=key)
reviews_data = response['Body'].read().decode('utf-8')

reviews = pd.read_csv(StringIO(reviews_data))

#extracting data from shipment fact table
client = boto3.client('s3')

bucket_name = '10alytics-capstone-bucket'
key ='orders_data/shipments_deliveries.csv'

response = client.get_object(Bucket=bucket_name,Key=key)
shipment_data = response['Body'].read().decode('utf-8')

shipment = pd.read_csv(StringIO(shipment_data))

In [3]:
#Samples of the fact tables:
display (orders.head(3))
print('*' * 60)
display (reviews.head(3))
print('*' * 60)
display (shipment.head(3))


,order_id,customer_id,order_date,product_id,unit_price,quantity,total_price
0,1,5,2022-07-13,24,139,10,1390
1,2,14,2021-04-06,2,273,4,1092
2,3,17,2022-07-29,20,253,9,2277


************************************************************


,review,product_id
0,1,21
1,3,1
2,2,8


************************************************************


,shipment_id,order_id,shipment_date,delivery_date
0,1,1,2022-07-14,NaN
1,2,2,NaN,NaN
2,3,3,2022-07-31,2022-08-03


### Connecting to Data warehouse

In [4]:
# Database Credentials
db_username = 'c5gp027946'
db_password = '**********'
db_id = 'c5gp027946'
db_host = '34.89.230.185'
db_port = 5432
db_name = 'd2b_accessment'
staging_schema = 'c5gp027946_staging'
analytics_schema = 'c5gp027946_analytics'

In [5]:
# Establish a connection using SQLAlchemy engine
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}/{db_name}')
engine.connect()

### Load the datasets into the Data warehouse

In [ ]:
orders.to_sql('orders', con = engine, if_exists='replace', index = False, schema = 'c5gp027946_staging')
reviews.to_sql('reviews', con = engine, if_exists='replace', index = False, schema = 'c5gp027946_staging')
shipment.to_sql('shipments_deliveries', con = engine, if_exists='replace', index = False, schema = 'c5gp027946_staging')

In [6]:
# Overview of the Date Table
dimdates = '''
SELECT * FROM if_common.dim_dates
Limit 3

'''
pd.read_sql(dimdates,engine)

,calendar_dt,year_num,month_of_the_year_num,day_of_the_month_num,day_of_the_week_num,working_day
0,2015-01-01,2015,1,1,4,False
1,2015-01-02,2015,1,2,5,True
2,2015-01-03,2015,1,3,6,False


#### Requirement 1

- The total number of orders placed on a public holiday every month, for the past year.

Note: A public holiday is a day with a day_of_the_week number in the range of 1 - 5 and a working_day value of false.

In [7]:
tt_order_hol_jan ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05'
)
select count(order_id) AS tt_order_hol_jan from CTE
WHERE month = 1

'''
pd.read_sql(tt_order_hol_jan,engine)

,tt_order_hol_jan
0,0


In [8]:
tt_order_hol_feb ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_feb from CTE
WHERE month = 2

'''
pd.read_sql(tt_order_hol_feb,engine)

,tt_order_hol_feb
0,17


In [9]:
tt_order_hol_mar ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_mar from CTE
WHERE month = 3

'''
pd.read_sql(tt_order_hol_mar,engine)

,tt_order_hol_mar
0,13


In [10]:
tt_order_hol_apr ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_apr from CTE
WHERE month = 4

'''
pd.read_sql(tt_order_hol_apr,engine)

,tt_order_hol_apr
0,12


may - dec

In [11]:
tt_order_hol_may ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_may from CTE
WHERE month = 5

'''
pd.read_sql(tt_order_hol_may,engine)

,tt_order_hol_may
0,0


In [12]:
tt_order_hol_jun ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_jun from CTE
WHERE month = 6

'''
pd.read_sql(tt_order_hol_jun,engine)

,tt_order_hol_jun
0,0


In [13]:
tt_order_hol_jul ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_jul from CTE
WHERE month = 7

'''
pd.read_sql(tt_order_hol_jul,engine)

,tt_order_hol_jul
0,13


In [14]:
tt_order_hol_aug ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_aug from CTE
WHERE month = 8

'''
pd.read_sql(tt_order_hol_aug,engine)

,tt_order_hol_aug
0,17


In [15]:
tt_order_hol_sep ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_sep from CTE
WHERE month = 9

'''
pd.read_sql(tt_order_hol_sep,engine)

,tt_order_hol_sep
0,0


In [16]:
tt_order_hol_oct ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_oct from CTE
WHERE month = 10

'''
pd.read_sql(tt_order_hol_oct,engine)

,tt_order_hol_oct
0,16


In [17]:
tt_order_hol_nov ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_nov from CTE
WHERE month =11

'''
pd.read_sql(tt_order_hol_nov,engine)

,tt_order_hol_nov
0,12


In [18]:
tt_order_hol_dec ='''

with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_dec from CTE
WHERE month = 12

'''
pd.read_sql(tt_order_hol_dec,engine)

,tt_order_hol_dec
0,0


#### Analysis of aggregated public holiday orders for the period (2021-09-06 - 2022-09-05)

In [19]:
agg_public_holiday ='''
SELECT '2022-09-05' AS ingestion_date, * 
FROM

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_jan from CTE
WHERE month = 1) as i,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_feb from CTE
WHERE month = 2) as ii,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_mar from CTE
WHERE month = 3)as iii,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_apr from CTE
WHERE month = 4) as iv,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_may from CTE
WHERE month = 5) as v,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_jun from CTE
WHERE month = 6) as vi,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_jul from CTE
WHERE month = 7) as vii,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_aug from CTE
WHERE month = 8) as viii,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_sep from CTE
WHERE month = 9) as ix,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_oct from CTE
WHERE month = 10) as x,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_nov from CTE
WHERE month = 11) as xi,

(with CTE as (
SELECT A.order_id, B.day_of_the_week_num as "day", EXTRACT(MONTH FROM B.calendar_dt) AS "month"
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_dates AS B
ON DATE(A.order_date) = B.calendar_dt
WHERE B.working_day ='False'
AND B.day_of_the_week_num IN (1,2,3,4,5)
AND B.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05')
select count(order_id) AS tt_order_hol_dec from CTE
WHERE month = 12) as xii

'''
df1=pd.read_sql(agg_public_holiday,engine)
df1
#ingestion_date||tt_order_hol_jan||tt_order_hol_feb||tt_order_hol_mar||tt_order_hol_apr||tt_order_hol_may||tt_order_hol_jun||tt_order_hol_jul||tt_order_hol_aug||tt_order_hol_sep||tt_order_hol_oct||tt_order_hol_nov||tt_order_hol_dec

,ingestion_date,tt_order_hol_jan,tt_order_hol_feb,tt_order_hol_mar,tt_order_hol_apr,tt_order_hol_may,tt_order_hol_jun,tt_order_hol_jul,tt_order_hol_aug,tt_order_hol_sep,tt_order_hol_oct,tt_order_hol_nov,tt_order_hol_dec
0,2022-09-05,0,17,13,12,0,0,13,17,0,16,12,0


#### Requirement 2

- Total number of late shipments

NB: A late shipment is one with shipment_date greater than or equal to 6 days after the
order_date and delivery_date is NULL Total number of undelivered shipments

NB: An undelivered shipment is one with delivery_date as NULL and shipment_date as NULL and the current_date 15 days after order_date.

In [20]:
late_shipments ='''
SELECT COUNT(*) as tt_late_shipments from
(SELECT E.shipment_date, D.order_date, (DATE(E.shipment_date) - DATE(D.order_date)) AS datediff
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND (DATE(E.shipment_date) - DATE(D.order_date)) >= 6) as subquery
'''

pd.read_sql(late_shipments,engine)

,tt_late_shipments
0,175


In [21]:
undelivered_items ='''
SELECT COUNT(*) as tt_undelivered_items FROM
(SELECT D.order_id,D.order_date, '2022-09-05' as "current_date", (Date('2022-09-05') - Date(D.order_date)) as "datediff"
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND E.shipment_date is null
AND (Date('2022-09-05') - Date(D.order_date)) = 15) AS subquery
'''
pd.read_sql(undelivered_items,engine)

,tt_undelivered_items
0,11


#### Analysis of aggregated shipments for the period (2021-09-06 - 2022-09-05)

In [22]:
agg_shipments ='''
SELECT '2022-09-05' AS ingestion_date, * 
FROM 

(SELECT COUNT(*) as tt_late_shipments from
(SELECT E.shipment_date, D.order_date, (DATE(E.shipment_date) - DATE(D.order_date)) AS datediff
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND (DATE(E.shipment_date) - DATE(D.order_date)) >= 6) as F) as a,

(SELECT COUNT(*) as tt_undelivered_items FROM
(SELECT D.order_id,D.order_date, '2022-09-05' as "current_date", (Date('2022-09-05') - Date(D.order_date)) as "datediff"
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND E.shipment_date is null
AND (Date('2022-09-05') - Date(D.order_date)) = 15) AS G) as b

'''
df2=pd.read_sql(agg_shipments,engine)
df2
#ingestion_date||tt_late_shipments||tt_undelivered_items

,ingestion_date,tt_late_shipments,tt_undelivered_items
0,2022-09-05,175,11


#### Requirement 3

- The product with the highest reviews, the day it was ordered the most, either that day was a public holiday, total review points, percentage distribution of the review points, and percentage distribution of early shipments to late shipments for that particular product.

In [23]:
star_performing_product ='''
with CTE as (
SELECT A.product_id, B.product_name, D.day_of_the_week_num,D.calendar_dt, C.review 
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05'
)
select product_name, calendar_dt as "most_ordered_day",sum(review) as "tt_review_points"
from CTE
GROUP BY product_name,calendar_dt
order by tt_review_points desc
limit 1

'''
pd.read_sql(star_performing_product,engine)

,product_name,most_ordered_day,tt_review_points
0,Hammer,2022-09-03,5256.0


In [24]:
public_holiday_indicator = '''

SELECT public_holiday
FROM (
SELECT *,

CASE
    WHEN working_day = 'False' 
    AND day_of_the_week_num IN (1, 2, 3, 4, 5) THEN 'Yes'
    ELSE 'No'
END AS Public_Holiday

FROM if_common.dim_dates
) subquery
WHERE calendar_dt = '2022-09-03'
'''

pd.read_sql(public_holiday_indicator,engine)

,public_holiday
0,No


In [25]:
one_star_review = '''
WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_one_star_review 
FROM Reviews
WHERE review = 1
'''
pd.read_sql(one_star_review,engine)

,pct_one_star_review
0,18.77


In [26]:
two_star_review = '''
WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_two_star_review 
FROM Reviews
WHERE review = 2
'''
pd.read_sql(two_star_review,engine)

,pct_two_star_review
0,17.33


In [27]:
three_star_review = '''
WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_three_star_review 
FROM Reviews
WHERE review = 3
'''
pd.read_sql(three_star_review,engine)

,pct_three_star_review
0,18.41


In [28]:
four_star_review = '''
WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_four_star_review 
FROM Reviews
WHERE review = 4
'''
pd.read_sql(four_star_review,engine)

,pct_four_star_review
0,19.86


In [29]:
five_star_review = '''
WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_five_star_review 
FROM Reviews
WHERE review = 5
'''
pd.read_sql(five_star_review,engine)

,pct_five_star_review
0,25.63


In [30]:
pct_early_shipment ='''
WITH TTS AS (
SELECT 
shipment_category, 
count(*) AS shipment_count,
count(*) * 100.0 / sum(count(*)) over() AS PCT
FROM (
SELECT E.shipment_date, D.order_date, (DATE(E.shipment_date) - DATE(D.order_date)) AS datediff,
CASE 
WHEN (DATE(E.shipment_date) - DATE(D.order_date)) >= 6 THEN 'late_shipment'
ELSE 'early_shipment'
END AS "shipment_category"
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND DATE(E.shipment_date) is not null
)subquery
GROUP BY shipment_category
)
SELECT pct AS pct_early_shipments
FROM TTS
WHERE shipment_category = 'early_shipment'

'''
pd.read_sql(pct_early_shipment,engine)

,pct_early_shipments
0,83.269598


In [31]:
pct_late_shipment ='''
WITH TTS AS (
SELECT 
shipment_category, 
count(*) AS shipment_count,
count(*) * 100.0 / sum(count(*)) over() AS PCT
FROM (
SELECT E.shipment_date, D.order_date, (DATE(E.shipment_date) - DATE(D.order_date)) AS datediff,
CASE 
WHEN (DATE(E.shipment_date) - DATE(D.order_date)) >= 6 THEN 'late_shipment'
ELSE 'early_shipment'
END AS "shipment_category"
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND DATE(E.shipment_date) is not null
)subquery
GROUP BY shipment_category
)
SELECT pct AS pct_late_shipments
FROM TTS
WHERE shipment_category = 'late_shipment'

'''
pd.read_sql(pct_late_shipment,engine)

,pct_late_shipments
0,16.730402


#### Analysis of best performing product in the period (2021-09-06 - 2022-09-05)

In [32]:
best_performing_product ='''
SELECT '2022-09-05' AS ingestion_date, * 
FROM 

(with CTE as (
SELECT A.product_id, B.product_name, D.day_of_the_week_num,D.calendar_dt, C.review 
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt BETWEEN '2021-09-06' AND '2022-09-05'
)
select product_name, calendar_dt as "most_ordered_day",sum(review) as "tt_review_points"
from CTE
GROUP BY product_name,calendar_dt
order by tt_review_points desc
limit 1) as R,

(SELECT public_holiday
FROM (
SELECT *,

CASE
    WHEN working_day = 'False' 
    AND day_of_the_week_num IN (1, 2, 3, 4, 5) THEN 'Yes'
    ELSE 'No'
END AS Public_Holiday

FROM if_common.dim_dates
) subquery
WHERE calendar_dt = '2022-09-03') as S,

(WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_one_star_review 
FROM Reviews
WHERE review = 1) as T,

(WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_two_star_review 
FROM Reviews
WHERE review = 2) as U,

(WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_three_star_review 
FROM Reviews
WHERE review = 3) as V,

(WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_four_star_review 
FROM Reviews
WHERE review = 4) as W,

(WITH Reviews AS (
SELECT B.product_name, C.review,
ROUND(count(*) * 100.0 / sum(count(*)) over(),2) AS pct
FROM c5gp027946_staging.orders AS A
JOIN if_common.dim_products AS B
ON A.product_id = B.product_id
JOIN c5gp027946_staging.reviews AS C
ON B.product_id = C.product_id
JOIN if_common.dim_dates AS D
ON D.calendar_dt = Date(A.order_date)
AND D.calendar_dt = '2022-09-03'
AND B.product_name like 'Hammer'
GROUP BY B.product_name, C.review
ORDER BY C.review desc
)
SELECT pct AS pct_five_star_review 
FROM Reviews
WHERE review = 5) as X,

(WITH TTS AS (
SELECT 
shipment_category, 
count(*) AS shipment_count,
count(*) * 100.0 / sum(count(*)) over() AS PCT
FROM (
SELECT E.shipment_date, D.order_date, (DATE(E.shipment_date) - DATE(D.order_date)) AS datediff,
CASE 
WHEN (DATE(E.shipment_date) - DATE(D.order_date)) >= 6 THEN 'late_shipment'
ELSE 'early_shipment'
END AS "shipment_category"
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND DATE(E.shipment_date) is not null
)subquery
GROUP BY shipment_category
)
SELECT pct AS pct_early_shipments
FROM TTS
WHERE shipment_category = 'early_shipment') as Y,

(WITH TTS AS (
SELECT 
shipment_category, 
count(*) AS shipment_count,
count(*) * 100.0 / sum(count(*)) over() AS PCT
FROM (
SELECT E.shipment_date, D.order_date, (DATE(E.shipment_date) - DATE(D.order_date)) AS datediff,
CASE 
WHEN (DATE(E.shipment_date) - DATE(D.order_date)) >= 6 THEN 'late_shipment'
ELSE 'early_shipment'
END AS "shipment_category"
FROM c5gp027946_staging.orders AS D
JOIN c5gp027946_staging.shipments_deliveries AS E
ON D.order_id = E.order_id
WHERE E.delivery_date is null
AND DATE(E.shipment_date) is not null
)subquery
GROUP BY shipment_category
)
SELECT pct AS pct_late_shipments
FROM TTS
WHERE shipment_category = 'late_shipment') as Z

'''

df3=pd.read_sql(best_performing_product,engine)
df3

,ingestion_date,product_name,most_ordered_day,tt_review_points,public_holiday,pct_one_star_review,pct_two_star_review,pct_three_star_review,pct_four_star_review,pct_five_star_review,pct_early_shipments,pct_late_shipments
0,2022-09-05,Hammer,2022-09-03,5256.0,No,18.77,17.33,18.41,19.86,25.63,83.269598,16.730402


### Loading the Analytics into the schema

In [33]:
#df1.to_sql('agg_public_holiday', con = engine, if_exists='replace',index=False,schema ='c5gp027946_analytics')
#df2.to_sql('agg_shipments', con = engine, if_exists='replace',index=False,schema ='c5gp027946_analytics')
#df3.to_sql('best_performing_product', con = engine, if_exists='replace',index=False,schema ='c5gp027946_analytics')

### Close connection to database

In [ ]:
conn.close()

### Uploading the best_performing_product into Datalake(s3)

In [ ]:
client.upload_file(Filename="C:/Users/USER/OneDrive/Desktop/analytics_export/grp02/best_performing_product.csv",
                   Bucket='10alytics-capstone-bucket',Key='grp02/best_performing_product.csv')

### Read from S3 Bucket to confirm the upload

In [34]:
my_bucket = s3_client.Bucket('10alytics-capstone-bucket')
obj_bucket = my_bucket.objects.all()
for i in obj_bucket:
    print(i)

s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='analytics_export/grp01/agg_public_holiday.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='analytics_export/grp01/agg_shipments.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='analytics_export/grp01/best_performing_product.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='analytics_export/grp02/agg_public_holiday.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='analytics_export/grp02/agg_shipments.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='analytics_export/grp02/best_performing_product.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='orders_data/')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='orders_data/orders.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='orders_data/reviews.csv')
s3.ObjectSummary(bucket_name='10alytics-capstone-bucket', key='orders_data/ship

END